In [1]:
import pandas as pd
import mysql.connector
import numpy as np
from sklearn import tree

def import_data():
    con  = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="000012300"
        )
    cur = con.cursor()
    cur.execute("USE mini_data")
    cur.execute("SELECT code from symptoms ORDER BY code")
    ##variables
    diseaseCodes = []
    symptomCode = []
    relations=[]

    for i in cur:
        symptomCode.append(i[0])

    cur.execute("SELECT code from diseases ORDER BY code")
    for i in cur:
        diseaseCodes.append(i[0])

    relations =  np.zeros((len(diseaseCodes), len(symptomCode)))

    for i in diseaseCodes:
        cur.execute("SELECT symptom_code from disease_symptom WHERE disease_code= '"+i+" '")
        for k in cur:
            relations[diseaseCodes.index(i)][symptomCode.index(k[0])]=1
        
    # for i in range(len(diseaseCodes)):
    #     print(diseaseCodes[i],end="\t")
    #     for j in range(len(symptomCode)):
    #         print(relations[i][j],end =" ")
    return diseaseCodes,symptomCode,relations

In [14]:
from sklearn.tree.export import export_text
diseaseCodes,symptomCode,relations = import_data()
clf = tree.DecisionTreeClassifier(criterion='entropy', splitter='best')
clf = clf.fit(relations,diseaseCodes)

print(clf.predict([relations[10]]))
print(diseaseCodes[10])

etx = export_text(clf,feature_names=symptomCode)
print(etx)

['C0004096']
C0004096
|--- C0392680 <= 0.50
|   |--- C0030193 <= 0.50
|   |   |--- C0015967 <= 0.50
|   |   |   |--- C0085631 <= 0.50
|   |   |   |   |--- C0241526 <= 0.50
|   |   |   |   |   |--- C0000737 <= 0.50
|   |   |   |   |   |   |--- C0043096 <= 0.50
|   |   |   |   |   |   |   |--- C0010200 <= 0.50
|   |   |   |   |   |   |   |   |--- C1299586 <= 0.50
|   |   |   |   |   |   |   |   |   |--- C0006625 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- C0015672 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |   |--- C0015672 >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: C0030312
|   |   |   |   |   |   |   |   |   |--- C0006625 >  0.50
|   |   |   |   |   |   |   |   |   |   |--- C0859032 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: C0005001
|   |   |   |   |   |   |   |   |   |   |--- C0859032 >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: C0034067
